In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

24/10/07 19:41:04 WARN Utils: Your hostname, joao-santos-desktop resolves to a loopback address: 127.0.1.1; using 192.168.1.70 instead (on interface wlp7s0)
24/10/07 19:41:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/07 19:41:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json("../../datalake/twitter_datascience")

In [5]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{146822630164340...|{[{2021-12-07T14:...|{1597742169376579...|  2022-11-29|
|[{935446829730291...|{[{2017-11-28T09:...|{1597741505149808...|  2022-11-29|
|[{140333728412517...|{[{2021-06-11T13:...|{1597741130573123...|  2022-11-29|
|[{131725596376263...|{[{2020-10-17T00:...|{1597740187773435...|  2022-11-29|
|[{942496609434292...|{[{2017-12-17T20:...|{1597739801792348...|  2022-11-29|
|[{190097582, 1597...|{[{2010-09-13T02:...|{1597739085149270...|  2022-11-29|
|[{156528072615680...|{[{2022-09-01T10:...|{1597738629081743...|  2022-11-29|
|[{145057394, 1597...|{[{2010-05-18T01:...|{1597738485531934...|  2022-11-29|
|[{17977727, 15977...|{[{2008-12-08T23:...|{1597737850904416...|  2022-11-29|
|[{19664316, 15977...|{[{2009-01-28T17:...|{1597737492081512...|

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |--

In [8]:
from pyspark.sql import functions as f

In [9]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [10]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{1468226301643403...|
|{8193148897894932...|
|{1317255963762634...|
|{1497937787181363...|
|{1468226301643403...|
|{1468226301643403...|
|{1220990287234969...|
|{1459140443954978...|
|{25204382, 159774...|
|{1011817655957893...|
|{9354468297302917...|
|{190097582, 15977...|
|{9354468297302917...|
|{142255858, 15977...|
|{1142424032794406...|
|{7101237361757839...|
|{2415478665, 1597...|
|{44894396, 159774...|
|{190097582, 15977...|
|{9354468297302917...|
+--------------------+
only showing top 20 rows



In [11]:
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [12]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [13]:
tweet_df.show(5)

+-------------------+-------------------+--------------------+-------------------+----------+-----------+-----------+-------------+--------------------+
|          author_id|    conversation_id|          created_at|                 id|like_count|quote_count|reply_count|retweet_count|                text|
+-------------------+-------------------+--------------------+-------------------+----------+-----------+-----------+-------------+--------------------+
|1468226301643403271|1597742169376579584|2022-11-29T23:59:...|1597742169376579584|         0|          0|          0|           98|RT @ratingology: ...|
| 819314889789493248|1597742113441341441|2022-11-29T23:59:...|1597742113441341441|         0|          0|          0|            2|RT @Datascience__...|
|1317255963762634752|1597742042284978176|2022-11-29T23:59:...|1597742042284978176|         0|          0|          0|            1|RT @genericgranol...|
|1497937787181363202|1597742024555274242|2022-11-29T23:59:...|1597742024555274242|

In [17]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [18]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [19]:
user_df.show(5)

+--------------------+-------------------+--------------------+--------------+
|          created_at|                 id|                name|      username|
+--------------------+-------------------+--------------------+--------------+
|2021-12-07T14:30:...|1468226301643403271|        Rakesh Kumar|      kasavsam|
|2017-01-11T22:47:...| 819314889789493248|Parody Aaron Sciv...|  Eintsein1980|
|2020-10-17T00:08:...|1317255963762634752|            Omua Mua|4constellation|
|2022-02-27T14:13:...|1497937787181363202|       MarkKelvin_44| MarkKelvin_44|
|2020-01-25T08:42:...|1220990287234969600|      micopia.online| micopiaonline|
+--------------------+-------------------+--------------------+--------------+
only showing top 5 rows



In [20]:
tweet_df.coalesce(1).write.mode("overwrite").json("output/tweet")
user_df.coalesce(1).write.mode("overwrite").json("output/user")
